<a href="https://colab.research.google.com/github/darisoy/EE517_Sp21/blob/master/EE517_HW2_distilbert_base_uncased_finetuned_sst_2_english.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Some useful links (delete when submitting lols):
    
https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification

https://github.com/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb

In [ ]:
pip install transformers

     |████████████████████████████████| 2.1MB 7.9MB/s 
     |████████████████████████████████| 3.3MB 38.1MB/s 
     |████████████████████████████████| 901kB 55.1MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import math
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
# check the available device
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cuda


### Load and check the data

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/SST-2/train.tsv', delimiter='\t')
df_dev = pd.read_csv('/content/drive/MyDrive/SST-2/dev.tsv', delimiter='\t')
df_test = pd.read_csv('/content/drive/MyDrive/SST-2/test.tsv', delimiter='\t')

In [ ]:
print("Training data: ", len(df_train))
print("Dev data: ", len(df_dev))
print("Test data: ", len(df_test))

Training data:  67349
Dev data:  872
Test data:  1821


### Load the model

In [ ]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = DistilBertModel.from_pretrained(model_name)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

### Preprocessing before sending to the model

In [ ]:
def get_max_len(tokenized):
    max_len = 0
    for i in tokenized.values:
        if len(i) > max_len:
            max_len = len(i)
    return max_len

In [ ]:
# This turns every sentence into the list of ids
tokenized_train = df_train['sentence'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized_dev = df_dev['sentence'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))


In [ ]:
# Padding
max_len = max(get_max_len(tokenized_train), get_max_len(tokenized_dev))
padded_train = np.array([i + [0]*(max_len-len(i)) for i in tokenized_train.values])
padded_dev = np.array([i + [0]*(max_len-len(i)) for i in tokenized_dev.values])

print("Padded shape (train): ", np.array(padded_train).shape)
print("Padded shape (dev): ", np.array(padded_dev).shape)

Padded shape (train):  (67349, 66)
Padded shape (dev):  (872, 66)


In [ ]:
# Tell BERT to ignore padding
attention_mask_train = np.where(padded_train != 0, 1, 0)
attention_mask_dev = np.where(padded_dev != 0, 1, 0)

In [ ]:
label_train_orig = df_train['label'].to_numpy()
label_dev_orig = df_dev['label'].to_numpy()

In [ ]:
# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(padded_train), torch.from_numpy(attention_mask_train), torch.from_numpy(label_train_orig))
valid_data = TensorDataset(torch.from_numpy(padded_dev), torch.from_numpy(attention_mask_dev), torch.from_numpy(label_dev_orig))

# dataloaders
batch_size = 64

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)

In [ ]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_mask_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample input mask size: ', sample_mask_x.size()) # batch_size, seq_length
print('Sample input mask: \n', sample_mask_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([64, 66])
Sample input: 
 tensor([[  101,  7537,  2000,  ...,     0,     0,     0],
        [  101,  3383,  1996,  ...,     0,     0,     0],
        [  101,  6414, 13675,  ...,     0,     0,     0],
        ...,
        [  101,  2049, 20957,  ...,     0,     0,     0],
        [  101,  2000,  4133,  ...,     0,     0,     0],
        [  101,  9020,  2000,  ...,     0,     0,     0]])

Sample input mask size:  torch.Size([64, 66])
Sample input mask: 
 tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

Sample label size:  torch.Size([64])
Sample label: 
 tensor([1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1,
        1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,
        1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1])


### Using BERT to encode

In [ ]:
def get_embeddings(target_loader):

    model.eval()
    model.to(device)

    all_embed = []
    all_label = []

    for sample, sample_mask, label in target_loader:
        with torch.no_grad():
            sample, sample_mask = sample.to(device), sample_mask.to(device)
            last_hidden_states = model(sample, attention_mask=sample_mask)
            # get all sentences, all hidden unit outputs, at [CLS] (if output hidden state)
            all_embed.append(last_hidden_states[0][:,0,:])
            all_label.append(label)

    return torch.cat(all_embed,dim =0), torch.cat(all_label)

In [ ]:
train_embed, train_label = get_embeddings(train_loader)
dev_embed, dev_label = get_embeddings(valid_loader)

In [ ]:
train_embed = train_embed.cpu().numpy()
dev_embed = dev_embed.cpu().numpy()
print("train features and labels: ", train_embed.shape, train_label.shape)
print("dev features and labels: ", dev_embed.shape, dev_label.shape)

train features and labels:  (67349, 768) torch.Size([67349])
dev features and labels:  (872, 768) torch.Size([872])


### Logistic Regression Classifier on top of BERT

In [ ]:
# Define Logistic Regression model
lr_model = LogisticRegression(random_state=0, solver='liblinear')

# Fit the model to training data
lr_model.fit(train_embed, train_label)

# make prediction using the trained model
y_train_pred = lr_model.predict(train_embed)
y_val_pred = lr_model.predict(dev_embed)

In [ ]:
train_accuracy = accuracy_score(train_label, y_train_pred)
val_accuracy = accuracy_score(dev_label, y_val_pred)

print("Training accuracy:", train_accuracy)
print("Validation accuracy:", val_accuracy) 

Training accuracy: 0.9908387652377912
Validation accuracy: 0.8979357798165137
